In [1]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd
import pickle

In [ ]:
# model = Word2Vec.load("../models/b25-sn-v256/b25-sn-v256.model")
model = Word2Vec.load("../models/b25-sn-v512/b25-sn-v512-b.model")

In [2]:
from algo_testing import son2vecgl_a

with open(
    "../models/b25-sn-v512/b25-sn-v512-g.pkl", "rb"
) as file:  # "rb" means read in binary mode
    model = pickle.load(file)

In [3]:
with open("../data/tokenized_data/playlist_names/dataset_test.pkl", "rb") as f:
    tokenized_playlists = pickle.load(f)

In [ ]:
print(model.nearest_k1(tokenized_playlists[0][0]))

In [ ]:
tested = 0
correct = 0

test_set = tokenized_playlists[:100]
a = 0
for playlist in test_set:
    a += len(playlist)
print(f"Total songs: {a}")

vgl_a = 0
vgl_b = 0

for playlist in test_set:
    for song in playlist:
        vgl_a += 1

        # try:

        # similar_words = model.wv.most_similar(song, topn=1)
        # similar_words = model.nearest(song, k=1)
        similar_words = model.nearest(song, k=1)
        if similar_words == []:
            continue

        tested += 1

        if any(word[0] in playlist for word in similar_words):
            correct += 1
        # except:
        #     vgl_b += 1
        #     continue


print(f"correct: {correct}")
print(f"Accuracy: {correct/tested}")
print(f"Tested: {vgl_a}")
print(f"Wrong: {vgl_b}")

# alpha = 0.8 --> 0.014339622641509434 accuracy
# alpha = 0.5 --> 0.014339622641509434 accuracy
# alpha = 0.015 --> 0.013584905660377358 accuracy
# alpha = 0.015 epoch 15 --> 0.055304740406320545 accuracy
# alpha = 0.015 epoch 30 --> 0.056 --> 15 epoch where it konverges
# need to change something about the algorithm...
# alpha = 0.015 epoch 30 + fuzzy testing (k=5) --> Accuracy: 0.10533333333333333
# the algorithm somewhat shows it's going in the right direction but needs an other approche
# alpha = 0.015 epoch 15 + trained with the whole dataset --> Accuracy (over 50 playlists): 0.21962264150943397
# alpah = 0.025 epoch 15 + whole set --> Accuracy: 0.23622641509433961
# alpah = 0.025 epoch 15 + whole set + included L2 Norm as score --> Accuracy: 0.052830188679245285


# gloVe type algo:
# alpha = 0.025 epoch 15 *1000 playlists vector_size = 512 --> Accuracy: 0.009505703422053232

Total songs: 3430
correct: 15
Accuracy: 0.009505703422053232
Tested: 3430
Wrong: 0


In [ ]:
test_set = tokenized_playlists[:100]

# Function to calculate Precision@K
def precision_at_k(model, test_playlists, k):
    """
    Calculates Precision@K for a recommendation model.

    Args:
        model: The trained recommendation model with a predict method.
        test_playlists (list of lists): A list of playlists, where each playlist is a list of ground truth songs.
        k (int): The number of top recommendations to consider.

    Returns:
        float: The average Precision@K across all test playlists.
    """
    precision_scores = []

    for playlist in test_playlists:
        # Exclude one song from the playlist as the "input context"
        input_context = playlist[:-1]
        ground_truth_song = playlist[-1]

        # Get model predictions (ranked list of recommended songs)
        predictions = model.predict(input_context)

        # Take the top-K predictions
        top_k_predictions = predictions[:k]

        # Check if the ground truth song is in the top-K predictions
        relevant_items = [song for song in top_k_predictions if song in playlist]
        precision = len(relevant_items) / k
        
        precision_scores.append(precision)

    # Return the average Precision@K
    return sum(precision_scores) / len(precision_scores)

# Example usage
# Assuming `model` is your trained recommendation model
# and `test_playlists` is a list of playlists from the test set
k = 10
precision_k = precision_at_k(model, test_set, k)
print(f"Precision@{k}: {precision_k}")


# Second Test with the Combined Artist and Trackname 
to exclude songs named the same with different vibe

#### b25-sn-v50 param: vector_size=50, window=5, min_count=1, sg=0
Accuracy: 0.36720959719106605

#### b25-sn-v256-a param: vector_size=256, window=5, min_count=1, sg=0
Accuracy: 0.36691699990246757

#### b25-sn-v256-b param: vector_size=256, window=10, min_count=1, sg=0
Accuracy: 0.43333658441431777

#### b25-sn-v256-c param: vector_size=256, window=20, min_count=1, sg=0
Accuracy: 0.4426996976494684

#### b25-sn-v256-d param: vector_size=256, window=20, min_count=1, sg=1, ns_exponent=0.0
Accuracy: 0.6513215644201698

#### b25-sn-v512-a param: vector_size=512, window=100, min_count=1, sg=0
Accuracy: 0.5702721154783965

#### b25-sn-v512-b param: vector_size=512, window=100, min_count=1, sg=1, ns_exponent=0.0
Accuracy: 0.773919828342924

#### b25-sn-v512-d - CBOS param: vector_size=512, epochs=5, learning_rate=0.015*

Accuracy: 0.08009367681498829

#### b25-sn-v512-e param: vector_size: 512, window=inf, min_count=1, epochs=15, learning_rate=0.025, algo=CBOS-d
Accuracy: 0.052830188679245285
* Used distance to add a score to the gradient

#### b25-snv128-f param: vector_size: 128, window=inf, min_count=1, epochs=15, learning_rate=0.025, algo=GLSE
Accuracy: 0.0038022813688212928



\* trained with only 3.5% of the training data 

## Unit tested Results

#### b25-sn-v50 param: vector_size=50, window=5, min_count=1, sg=0
Accuracy: 0.260704184141227

#### b25-sn-v256-a param: vector_size=256, window=5, min_count=1, sg=0
Accuracy: 0.28089339705452065

#### b25-sn-v256-b param: vector_size=256, window=10, min_count=1, sg=0
Accuracy: 0.3198088364381157

#### b25-sn-v256-c param: vector_size=256, window=20, min_count=1, sg=0
Accuracy: 0.39529893689651807

#### b25-sn-v256-d param: vector_size=256, window=20, min_count=1, sg=1, ns_exponent=0.0
Accuracy: 0.4845411099190481

#### b25-sn-v512-a param: vector_size=512, window=100, min_count=1, sg=0
Accuracy: 0.5000487662147665

#### b25-sn-v512-b param: vector_size=512, window=100, min_count=1, sg=1, ns_exponent=0.0
Accuracy: 0.6720959719106603




## Old not Accurate Results...

In [ ]:
# first model b25-sn-v256: just the base data with no tweaks of the data and the most basic var of the model
# 0.07389817629179331 accuracy over 100 playlists is very low as far as i feel how good the suggested songs are from the model
# don't know if the number will rise with more playlist but the code needs more optimizing

# second model b25-sn-v256-b: training it with a bigger dataset - base is the normal dataset but extended with shuffeld playlists on top
# 0.2773556231003039 accuracy over 100 playlist - thats a solid score if you think about every 1/4 songs that is suggested is correct
# decent result for a second draft

# third model b25-sn-v256-c: based on model the data of b25-sn-v256-b but the context window is now 20 not 5
# 0.48822188449848025 accuracy over 100 playlists thats a good score for songs suggestion

# b25-sn-v256-d: reduced training data to 80% of the set to have a better testing without the chance of overfitting
# 0.21337249481686246 accuracy over 100 playlists - was to be anticipated because of reducing the training set and the split of train and test data
# still need to test all models against a way bigger chunck of the test set
# before splitting test and train the data should be shuffeld to remove bias of the data to a specific section

# tested b25-sn-v256-c again but with only the first pick beeing good or bad
# 0.37232204561161025 is still a solide score for a normal test without mostly any biase

# b25-sn-v256-f bigger vector of 512 and a bigger window of 100 while training
#  accuracy

# comments about the testing
# still need to test them against way more playlists not only 100
# also need to cut between training and test data to prevent overfitting the model to the values i just want to here
# it would be interesting what accuracy is possible to reach with this approach
# could also extend the context window
# also some basic data exploration needs to be done

In [ ]:
# just some manuel testing of the model

song = "good kid".lower()

matches = [key for key in model.wv.key_to_index.keys() if song in str(key).lower()]
print(matches)